# 必要なパッケージ・データの読み込み

## 準備：必要なパッケージの読込

In [ ]:
### 必要なパッケージ（ライブラリ）の読み込み ###
import pandas as pd             # データフレーム型変数を取り扱えるパッケージ"pandas"を読込み（以降"pd"と略記）
import matplotlib.pyplot as plt # グラフ描画のパッケージ"matplotlib"を読込み（以降"plt"と略記）
import seaborn as sns           # 上記matplotlibをベースにした高機能可視化モジュール"saeborn"を読込み（以降"sns"と略記）
import numpy as np              # 数値演算のためのパッケージ"numpy"を読込み（以降"np"と略記）

## データの理解（観察）

### データ読込

In [ ]:
### データの読み込み ###
df = pd.read_excel('ファイル名入力（Tabキーによる文字補完を活用）')

# 読み込むファイル名：演習04_POSデータ.xlsx
# ※ "演習03_" まで入力し、TABキーで補完するのが効率的

In [ ]:
df  # 顧客ごと（0~4）に何を購入したかが列方向に格納（e.g. 0番の顧客は、パンと牛乳を同時購入）

### グラフによる可視化

In [ ]:
### ヒストグラムによる傾向確認 ###

df.stack().hist()
        # df.stack() で、複数列のデータを1列に集約（縦方向に連結）
        # df.hist() で、簡易的なヒストグラム描画

# 日本語文字化け対策
plt.rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao',
                                   'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']
# グラフ描画
plt.show()

### 参考：描画設定
※重なりなどの描画崩れや、文字化けなど、描画がうまくいかない場合

In [ ]:
### 以下、描画の細かいデザイン設定 ###

# 出力されるグラフ画像の解像度を上げる
%config InlineBackend.figure_formats = {'png', 'retina'}

# グラフのサイズ指定
# plt.rcParams['figure.figsize'] = 10, 5

# 文字化け対策にフォント指定（Win/Macなどの環境に依らず、横断的に設定）
plt.rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao',
                                   'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']

# フォントサイズ一括指定
# plt.rcParams['font.size'] = 14

## 参考：Anaconda Promptでのアソシエーション分析パッケージインストール
※インストール済みの方は対応不要

In [ ]:
### Anaconda Promptでのアソシエーション分析パッケージ "mlxtend" インストール ###
# （※Macの場合はターミナル上からでOK）

# 下記コマンドをコピペして使用
# conda install -c conda-forge mlxtend

# ※どうしても上記でエラーが出てうまくいかない場合は、 pip install mlxtend を用いる
#.  ただし、conda install と pip install は相性が悪い部分もあり、公式でも非推奨

## データの準備（加工）

### 欠損値処理

#### 欠損値の確認

In [ ]:
### 欠損値の確認 ###
df.isnull().sum()

#### 欠損値の補完

In [ ]:
# 特になし

#### 欠損値の再確認

In [ ]:
# 特になし

### One-Hotベクトル表現への変換

In [ ]:
### アソシエーション分析の入力データフォーマットである "One-Hotベクトル"表記※ に変換
# ※行方向に顧客、列方向にアイテム、各セルには出現頻度が格納
# （変換プロセスは後述を参照）

df_onehot = df.stack().str.get_dummies().groupby(level=0).sum()
display(df_onehot)

### （参考）One-Hotベクトルへの変換プロセス確認

#### データの「縦持ち」

In [ ]:
### dfのデータを列ごとに切り出して、縦方向に連結（1列に集約） ###
df.stack()

#### 頻度集計

In [ ]:
### 集約列に含まれる各要素（単語）をカラム名として抽出し、###
### 各要素の出現頻度をマトリックス形式で集計              ###
df.stack().str.get_dummies()

#### One-Hotベクトル表現

In [ ]:
df

In [ ]:
### 顧客ごと（元の1レコードごと）に頻度集計をとる ###
df.stack().str.get_dummies().groupby(level=0).sum()
# groupby()は集計関数
# オプション level=0 で、複数のインデックスのうち一番外側のインデックスラベルを用いて集計
# sum()でその合計を取得

## モデルの構築

### 必要なパッケージなどのインポート

In [ ]:
### アソシエーション分析に必要なaprioriクラスをインポート ### 
from mlxtend.frequent_patterns import apriori

### モデル構築

In [ ]:
### アソシエーション分析モデル構築（アプリオリ実行）###
frequent_itemsets = apriori( df_onehot, min_support=0.4, use_colnames=True )

                        # df_oneohot: 入力データ（one-hotベクトル形式で与える必要あり）
                        # min_support=0.4: 最小の支持度（全体の中での同時購入率）を0.4として設定（出力されるルールが多すぎてしまうため）
                        # use_colnames=True: 出力結果に、番号ではなく実際のカラム名（製品名）を出力する

### 結果出力

In [ ]:
### 指定した支持度以上の商品組み合わせを出力 ###
display ( frequent_itemsets )

In [ ]:
### 支持度の大きい順に並び替え（ソート） ###
display ( frequent_itemsets.sort_values(['support'], ascending=False) )

## モデルの評価

### 各ルールの信頼度・リフト値出力

#### 確信度（Confidence）上位ランキング

In [ ]:
### 各ルールの信頼度/リフト値算出に必要なaprioriassociation_rulesクラスをインポート ### 
from mlxtend.frequent_patterns import association_rules

rules1 = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.7)
                        # frequent_itemsets: 入力データ（アプリオリにより抽出された最小支持度以上の商品組み合わせ）
                        # metric="confidence": 出力の閾値を設定する指標
                        # min_threshold=0.7:   閾値

In [ ]:
### 抽出されたルールの出力 ###
display( rules1.sort_values(['confidence'], ascending=False) )

#### リフト値上位ランキング

In [ ]:
rules2 = association_rules(frequent_itemsets, metric='lift', min_threshold=1.0)
                        # frequent_itemsets: 入力データ（アプリオリにより抽出された最小支持度以上の商品組み合わせ）
                        # metric="lift":     出力の閾値を設定する指標
                        # min_threshold=0.7: 閾値
display( rules2.sort_values(['lift'], ascending=False) )

In [ ]:
## モデルの評価

### 各ルールの信頼度・リフト値出力

#### 確信度（Confidence）上位ランキング

### 各ルールの信頼度/リフト値算出に必要なaprioriassociation_rulesクラスをインポート ### 
from mlxtend.frequent_patterns import association_rules

rules1 = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.7)
                        # frequent_itemsets: 入力データ（アプリオリにより抽出された最小支持度以上の商品組み合わせ）
                        # metric="confidence": 出力の閾値を設定する指標
                        # min_threshold=0.7:   閾値

### 抽出されたルールの出力 ###
display( rules1.sort_values(['confidence'], ascending=False) )

#### リフト値上位ランキング

rules2 = association_rules(frequent_itemsets, metric='lift', min_threshold=1.0)
                        # frequent_itemsets: 入力データ（アプリオリにより抽出された最小支持度以上の商品組み合わせ）
                        # metric="lift":     出力の閾値を設定する指標
                        # min_threshold=0.7: 閾値
display( rules2.sort_values(['lift'], ascending=False) )